In [1]:
# install the latest version of requests
!pip install requests==v2.21.0

In [2]:
import requests
requests.__version__

'2.21.0'

In [0]:
# configure a generical header
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/vnd.geo+json, application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}


In [4]:
!pip install git+https://github.com/wrobstory/vincent.git

  Cloning https://github.com/wrobstory/vincent.git to /tmp/pip-req-build-bal6wtt_
  Running command git clone -q https://github.com/wrobstory/vincent.git /tmp/pip-req-build-bal6wtt_
  Stored in directory: /tmp/pip-ephem-wheel-cache-3z58wh90/wheels/39/e6/e6/41c0e1148e82d6a9dfb2b0eff0d72a545a825fedf20635fead
Successfully built vincent


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from branca.colormap import linear
import numpy as np
import vincent

In [6]:
# Install the latest release
!pip install folium==0.8.2

     |████████████████████████████████| 92kB 3.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.2 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [7]:
# check the version installed
import folium
folium.__version__

'0.8.2'

In [0]:
estados = pd.DataFrame()
municipios = pd.DataFrame()

estadosRequest = False

# Dados do ultimo mapa de estado gerado
malha = {}
dataFrameEstado = pd.DataFrame()
dictRanking = {}
titulo = ""

# Malha do Brasil a nivel de estado
url = "https://servicodados.ibge.gov.br/api/v2/malhas/?resolucao=2&formato=application/vnd.geo+json"

response = requests.get(url, headers=headers)

brasil_data = response.json()

In [0]:
def fillEstadosData():
  response = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados",
                       headers=headers)
  estadosJson = response.json()
  
  ids = []
  siglas = []
  nomes = []
  
  for estado in estadosJson:
    ids.append(str(estado["id"]))
    siglas.append(str(estado["sigla"]))
    nomes.append(str(estado["nome"]))

  global estados  
  estados["Id"] = ids
  estados["Sigla"] = siglas
  estados["Nome"] = nomes
  estados.index = estados["Id"]
  estadosRequest = True

def fillMunicipiosData(estadoNumero):
  response = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/"+ str(estadoNumero) +"/municipios",
                       headers=headers)
  municipiosJson = response.json()
  
  ids = []
  nomes = [] 
  
  for municipio in municipiosJson:
    ids.append(str(municipio["id"]))
    nomes.append(str(municipio["nome"]))
    
  global municipios
  
  municipios = pd.DataFrame()
    
  municipios["Id"] = ids
  municipios["Nome"] = nomes
  municipios.index = municipios["Id"]

In [0]:
def getEstadoNumero(estado):
  
  if( not estadosRequest):
    fillEstadosData()
  sigla = estados.loc[estados.Sigla == estado]
  estadoNumero = '0'
  if( not sigla.empty):
    estadoNumero = sigla.Id[0]
  
  nome = estados.loc[estados.Nome == estado]
  if( not nome.empty):
    estadoNumero = nome.Id[0]
  
  return str(estadoNumero)

def getNumeroLocal(estado, municipio=""): 
  estadoNumero = getEstadoNumero(estado)
  
  if(municipio != ""):
    fillMunicipiosData(estadoNumero)
    nome = municipios.loc[municipios.Nome == municipio]
    if( not nome.empty):
      return nome.Id[0]
  else:
    return estadoNumero
  
  print("Não encontramos o local indicado, talvez o nome esteja um pouco diferente nos dados do IBGE")
  return -1

In [0]:
def localId2LocalName(ibgeId): 
  if(len(ibgeId) == 2):
    if(ibgeId == "BR"):
      return "todo o Brasil"
    if(estados.empty):
      fillEstadosData()
    data = estados.loc[estados.Id == (ibgeId)]
    nome = data.Nome[0]
    return nome
  
  url = "http://servicodados.ibge.gov.br/api/v1/localidades/"
  
  if(len(ibgeId) == 7):
    url += "municipios/" + ibgeId
    
  response = requests.get(url, headers=headers)
  json_data = response.json()
  return json_data["nome"]
  
  
def mapForState(nome, sexo, estado):
  
  title = nome.upper()
  if(sexo == "M"):
    title += " para Homens"
  elif(sexo == "F"):
    title += " para Mulheres"
  title += " em " + estado
  
  global titulo
  global malha
  global dataFrameEstado
  
  if(title == titulo):
    return (dataFrameEstado,malha,title)
  
  number = getEstadoNumero(estado)
  fillMunicipiosData(number)
  
  sidra_url = "http://api.sidra.ibge.gov.br/values/t/1378/p/2010/v/93/c1/0/c2/0/c287/0/c455/0"
  
  url = "http://api.sidra.ibge.gov.br/values/t/1378/p/2010/v/93/c1/0/c2/0/c287/0/c455/0"
  count = 0
  
  municipios_id = []
  municipios_pop = []
  named_pop = []
  
  print("Obtendo dados dos municipios do Estado... Por favor aguarde deve demorar um bocado...")
  
  for municipio in municipios["Id"]:
    if(count == 8):
      #print(url)
      count = 0
      response = requests.get(url, headers=headers)
      url = sidra_url
      municipios_data = response.json()
      for item in municipios_data[1:]:
        municipios_id.append(str(item["D7C"]))
        municipios_pop.append(int(item["V"]))

        query = "?localidade="+str(item["D7C"])
        response = requests.get("https://servicodados.ibge.gov.br/api/v2/censos/nomes/"+nome+query,
                           headers=headers)
        json_data = response.json()
        if(json_data == []):
          named_pop.append(0)
          print("Dados de " + localId2LocalName(str(item["D7C"])) + " não encontrados... Exitem menos de 10 pessoas com esse nome lá")
        else:
          named_pop.append(int(json_data[0]["res"][-1]["frequencia"]))
          print("Dados de " + localId2LocalName(str(item["D7C"])) + " obtidos")
        
    url += "/n6/" + str(municipio)
    count += 1
  if(count != 0):
    #print(url)
    count = 0
    response = requests.get(url, headers=headers)
    url = sidra_url
    municipios_data = response.json()
    for item in municipios_data[1:]:
      municipios_id.append(str(item["D7C"]))
      municipios_pop.append(int(item["V"]))

      query = "?localidade="+str(item["D7C"])
      response = requests.get("https://servicodados.ibge.gov.br/api/v2/censos/nomes/"+nome+query,
                         headers=headers)
      json_data = response.json()
      if(json_data == []):
        named_pop.append(0)
      else:
        named_pop.append(int(json_data[0]["res"][-1]["frequencia"]))
  
  prop = (np.array(named_pop) / np.array(municipios_pop))
  
  municipio_df = pd.DataFrame.from_dict({"Localidade:":municipios_id,"População:":municipios_pop,
                                         "Frequencia:":named_pop,"Proporção:":prop})
  
  print("Dados do Estado obtidos, requisitando malha dos municipios...")
  
  response = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/" + str(number) + "?resolucao=5&formato=application/vnd.geo+json",
                           headers=headers)
  municipio_malha = response.json()
  
  municipio_df.index = municipios_id
    
  dataFrameEstado = municipio_df
  malha = municipio_malha
  titulo = title
  
  print("Malha obtida.")
  
  return (municipio_df, municipio_malha, title)
    
  
def generateBarhPlot(jsonData, ax):
  data = {}
  data["Local"] = jsonData["localidade"]
  data["Nome"] = jsonData["nome"]
  frequencia = []
  periodo = []
  
  for item in jsonData["res"]:
    periodo.append(item["periodo"])
    frequencia.append(item["frequencia"])
    
  dt = pd.DataFrame.from_dict({"Frequencia:":frequencia })
  dt.index = periodo
  
  title = "Frequencia do nome " + jsonData["nome"] 
  if(jsonData["sexo"] == "M"):
    title += " para Homens"
  if(jsonData["sexo"] == "F"):
    title += " para Mulheres"
  title += " em " + localId2LocalName(jsonData["localidade"]) + " por periodo"
  
  return dt.plot.barh(ax=ax, title=title)

def generateDataFrameAndTitle(jsonData, nome, query):
  localidade = []  
  populacao = []
  frequencia = []
  proporcao = []
  
  for item in jsonData:
    localidade.append(str(item["localidade"]))
    populacao.append(item["res"][0]["populacao"])
    frequencia.append(item["res"][0]["frequencia"])
    proporcao.append(round(item["res"][0]["proporcao"]/100000,6))
    
  dt = pd.DataFrame.from_dict({"Localidade:":localidade,"População:":populacao,"Frequencia:":frequencia,"Proporção:":proporcao})
  dt.index = localidade
  
  title = nome.upper()
  if(query.find("sexo=M") != -1):
    title += " para Homens"
  if(query.find("sexo=F") != -1):
    title += " para Mulheres"
  title += " no Brasil"
  
  global brasil_data
  global asd
  asd = dt
  return (dt, brasil_data, title)



In [0]:
from branca.colormap import linear

def placeDataInMap(df, malha):
  maior_porcentagem = {}
  first = True
  for feature in malha["features"]:
    ibge_id = str(feature['properties']['codarea'])
    feature["properties"]["Localidade"] = localId2LocalName(str(ibge_id))
    feature["properties"]["População"] = str(df.loc[ibge_id, "População:"])
    feature["properties"]["Frequência"] = str(df.loc[ibge_id, "Frequencia:"])
    feature["properties"]["Porcentagem"] = str(df.loc[ibge_id, "Proporção:"])
    feature['properties']['codarea'] = str(ibge_id)
    if(first):
      maior_porcentagem = feature["properties"]
      first = False
    else:
      if(feature["properties"]["Porcentagem"] > maior_porcentagem["Porcentagem"]):
        maior_porcentagem = feature["properties"] 
    return (df, malha)
  
def createMap(df, malha, title):
  # colormap yellow and green (YlGn)
  colormap = linear.Blues_03.scale(
      df["Proporção:"].min(),
      df["Proporção:"].max())

  colormap.caption=title
  
  print("Associando dados de cada região com sua posição no mapa. Isso deve demorar um pouco...")    
  
  maior_porcentagem = {}
  first = True
  df.index = pd.Series([str(x) for x in df["Localidade:"]])  
  for feature in malha["features"]:
    ibge_id = str(feature['properties']['codarea'])
    feature["properties"]["Localidade"] = localId2LocalName(str(ibge_id))
    if(ibge_id in df.index): 
      feature["properties"]["População"] = str(df.loc[ibge_id, "População:"])
      feature["properties"]["Frequência"] = str(df.loc[ibge_id, "Frequencia:"])
      feature["properties"]["Porcentagem"] = ( '%.9f' % df.loc[ibge_id, "Proporção:"])
    else:
      url = "http://api.sidra.ibge.gov.br/values/t/1378/p/2010/v/93/c1/0/c2/0/c287/0/c455/0/n3/11/n3/"+ibge_id
      response = requests.get(url, headers=headers)
      data = response.json()
      s = df.iloc[0]
      s.name = str(ibge_id)
      s["Proporção:"] = 0
      df = df.append(s)
      
      feature["properties"]["População"] = str(data[1]["V"])
      feature["properties"]["Frequência"] = "0"
      feature["properties"]["Porcentagem"] = "0"
    feature['properties']['codarea'] = str(ibge_id)
    if(first):
      maior_porcentagem = feature["properties"]
      first = False
    else:
      if(feature["properties"]["Porcentagem"] > maior_porcentagem["Porcentagem"]):
        maior_porcentagem = feature["properties"] 
        
  print("Gerando Mapa")
  m = folium.Map(
      location = [-15.77972, -47.92972],
      zoom_start = 4,
      # Outras opções: 'Stamen Toner'
      tiles = 'Stamen Terrain'
  )

  folium.GeoJson(malha,
                 name = title,
                 style_function = lambda x: {'fillColor': colormap(df.loc[x['properties']['codarea'],
                                                                             "Proporção:"]),
                                           'color': 'black','weight':2, 'fillOpacity':0.8},
                 tooltip=folium.GeoJsonTooltip(fields=['Localidade',"População","Frequência","Porcentagem"], 
                                              aliases=['Estado:',"População:","Frequência:","Porcentagem"], 
                                              localize=True)
                ).add_to(m)

    
  folium.Marker([maior_porcentagem["centroide"][1],maior_porcentagem["centroide"][0]],
                popup='<i>'+ maior_porcentagem["Localidade"] + " " + maior_porcentagem["Porcentagem"] +'</i>',
                tooltip="Maior Porcentagem").add_to(m)
  
  colormap.add_to(m)
  
  print("Mapa gerado")
  
  return m


def printRankingMap(ranking, malha, title): 
  print("Associando dados de cada região com sua posição no mapa. Isso deve demorar um pouco...")    
  
  for feature in malha["features"]:
    ibge_id = str(feature['properties']['codarea'])
    feature["properties"]["Localidade"] = localId2LocalName(str(ibge_id))
    #feature["properties"]["DataFrame"] = ranking[ibge_id]
    feature['properties']['codarea'] = str(ibge_id)

    
  print("Gerando Mapa")
  m = folium.Map(
      location = [-15.77972, -47.92972],
      zoom_start = 4,
      # Outras opções: 'Stamen Toner'
      tiles = 'Stamen Terrain'
  )

  folium.GeoJson(malha,
                 name = title,
                 #style_function = lambda x: {'fillColor': colormap(df.loc[x['properties']['codarea'],
                 #                                                            "Proporção:"]),
                 #                          'color': 'black','weight':2, 'fillOpacity':0.8},
                 tooltip=folium.GeoJsonTooltip(fields=['Localidade'], 
                                              aliases=['Estado:'], 
                                              localize=True)
                ).add_to(m)

  for feature in malha["features"]:
    ibge_id = str(feature['properties']['codarea'])
  
    # Create an object popup and adding a graph for it
    popup = folium.Popup(max_width=450)
    
    bar = vincent.Bar(ranking[ibge_id],height=300,width=400)
    
    ax = vincent.AxisProperties(
         labels = vincent.PropertySet(angle=vincent.ValueRef(value=45),
                                      align=vincent.ValueRef(value='left')))
    bar.axes[0].properties = ax
        
    folium.Vega(bar, 
                height=350, 
                width=450).add_to(popup)

    # Print a icon on map
    folium.Marker([feature['properties']["centroide"][1],feature['properties']["centroide"][0]], 
                  tooltip=feature['properties']["Localidade"],
                  popup=popup).add_to(m)
  
      
  print("Mapa gerado")
  
  return m

def RankingMap(sexo, estado):
  if(estado == ""):
      estadp = "BR"
  title = "Ranking de nomes"
  if(sexo == "M"):
    title += " para Homens"
  elif(sexo == "F"):
    title += " para Mulheres"
  title += " em " + estado
  
  global titulo
  global malha
  global dictRanking
  
  if(title == titulo):
    return printRankingMap(dictRanking,malha,title)
  
  global municipios
  global estados
  locais = []
  
  if(estado == ""):
    number = 0  
    fillEstadosData()
    locais = estados
  else:
    number = getEstadoNumero(estado)
    fillMunicipiosData(number)
    locais = municipios

  dictRanking = {}
  
  for local in locais["Id"]:
    query = "?localidade="+str(local)
    response = requests.get("https://servicodados.ibge.gov.br/api/v2/censos/nomes/ranking"+query, headers=headers)
    jsonData = response.json()

    jsonData = jsonData[0]

    data = {}
    data["Local"] = jsonData["localidade"]
    nome = []
    frequencia = []
    ranking = []

    for item in jsonData["res"]:
      nome.append(item["nome"])
      frequencia.append(item["frequencia"])

    dt = pd.DataFrame.from_dict({"Frequencia:":frequencia })
    dt.index = nome
    dictRanking[str(local)] = dt
    print("Dados de " + localId2LocalName(str(local)) + " obtidos")
  
  print("Dados dos locais obtidos, requisitando malha a ser usada...")
  
  if(estado == ""):
    response = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/?resolucao=2&formato=application/vnd.geo+json",
                             headers=headers)
  else:
    response = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/" + str(number) + "?resolucao=5&formato=application/vnd.geo+json",
                             headers=headers)
  
  malha = response.json()
  
  malha = malha
  titulo = title
  
  return printRankingMap(dictRanking,malha,title)

In [0]:
def MaisComuns(query, ax):
  response = requests.get("https://servicodados.ibge.gov.br/api/v2/censos/nomes/ranking"+query, headers=headers)
  jsonData = response.json()
  
  jsonData = jsonData[0]
  
  data = {}
  data["Local"] = jsonData["localidade"]
  nome = []
  frequencia = []
  ranking = []
  
  for item in jsonData["res"]:
    nome.append(item["nome"])
    frequencia.append(item["frequencia"])
    #ranking.append(item["ranking"])
    
  dt = pd.DataFrame.from_dict({"Frequencia:":frequencia })
  dt.index = nome
  
  title = "Ranking dos nomes"
  if(jsonData["sexo"] == "M"):
    title += " para Homens"
  if(jsonData["sexo"] == "F"):
    title += " para Mulheres"
  title += " em " + localId2LocalName(jsonData["localidade"])
  
  return dt.plot.barh(ax=ax, title=title)

def search(nome, query, Map, Ranking):
  global estado
  global sexo
  if(Ranking and Map and query.find("UF") == -1):
    return RankingMap(sexo, estado)
  
  if(Ranking and query.find("UF") == -1):
    fig, ax = plt.subplots(figsize=(10, 5))
    return MaisComuns(query, ax)
  
  if(Map and estado != "" and query.find("UF") == -1):
    data = mapForState(nome, sexo, estado)
    return createMap(data[0],data[1],data[2])
  
  response = requests.get("https://servicodados.ibge.gov.br/api/v2/censos/nomes/"+nome+query,
                       headers=headers)
  json_data = response.json()
  
  if(len(json_data) == 0):
    return "O nome indicado não foi encontrado nos dados do IBGE, segundo eles \"a quantidade mínima de ocorrências para que seja divulgado os resultados é de 10 por município, 15 por Unidade da Federação e 20 no Brasil.\""
  if(query.find("UF") != -1):
    data = generateDataFrameAndTitle(json_data, nome, query)
    return createMap(data[0],data[1],data[2])
  
  fig, ax = plt.subplots(figsize=(10, 5))
  
  return generateBarhPlot(json_data[0], ax)

In [14]:
# Digite aqui seu nome (Sem acentos e não pode ser nome composto, apenas uma unica palavra):
# Talvez a partir do censo de 2020 mais dados estejam disponiveis
meuNome = "enzo"

# Abaixo podemos especificar os parametros dos filtros opcionais, caso não queira algum deixe-o em branco

# Para especificar se queremos apenas homens (M) ou mulheres (F) com esse nome
sexo = ""

# Caso queira rapidamente saber a frequencia do nome em cada estado do Brasil (Anula qualquer outra busca)
UF = True

# Caso queira apenas o ranking dos nomes de um local, sem pesquisar por um nome em especifico
Ranking = False

# Caso queira o mapa relativo ao estado e seus municipios sobre um nome especifico (Especifique apenas o estado como local)
# Provavelmente deve demorar um pouco... Afinal exitem muitos municipios em cada estado...
Map = False


# Para especificar o local onde a busca será feita
# O Estado onde esse local está (Caso apenas ele seja especificado a busca será nele)
estado = ""
# A Nome do municipio
municipio = ""

localidadeNumero = 0
if(not UF):
  localidadeNumero = getNumeroLocal(estado, municipio)

query = "?"
querySexo = "sexo="
querylocalidade = "localidade="
queryUF = "groupBy=UF"
_and = "&"

finalQuery = query+querylocalidade+str(localidadeNumero)
if(UF):
  finalQuery = query+queryUF
if(sexo!=""):
  finalQuery += _and+querySexo+sexo 

search(meuNome, finalQuery, Map, Ranking)

Associando dados de cada região com sua posição no mapa. Isso deve demorar um pouco...
Gerando Mapa
Mapa gerado
